# Python Code to Transfer data to Globus

### Steps to install globus-sdk

1. Install Globus SDK with the help of pip package manager\
pip install globus-sdk

2. Create a new Project in the Globus developer portal\
https://auth.globus.org/v2/web/developers

3. Create a new application to get the Client ID

4. Select the scopes of your app as openid, email, profile\
urn:globus:auth:scope:transfer.api.globus.org:all

### Step 1: To get the access tokens

In [ ]:
import globus_sdk

Your_Client_ID = "<Client ID>" #Generate the client ID through Globus developer web portal

client = globus_sdk.NativeAppAuthClient(Your_Client_ID)
client.oauth2_start_flow()

authorize_url = client.oauth2_get_authorize_url()
print("Please go to this URL and login: {0}".format(authorize_url))

auth_code = input("Please enter the code you get after login here: ").strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

globus_auth_data = token_response.by_resource_server["auth.globus.org"]
globus_transfer_data = token_response.by_resource_server["transfer.api.globus.org"]

AUTH_TOKEN = globus_auth_data["access_token"]
TRANSFER_TOKEN = globus_transfer_data["access_token"]

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=%3CClient+ID%3E&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=2ZHYvCpZa0QlMGSNoZ9T4TY4Df-_JH8DV53k23Rf20A&code_challenge_method=S256&access_type=online


### Step 2: Use the access token to list out the endpoints

In [ ]:
authorizer = globus_sdk.AccessTokenAuthorizer(TRANSFER_TOKEN)
tc = globus_sdk.TransferClient(authorizer=authorizer)

print("My Local Endpoints:")
for ep in tc.endpoint_search(filter_scope="my-endpoints"):
    print("[{}] {}".format(ep["id"], ep["display_name"]))


### Step 3: Find a specific endpoint which is shared with you

In [ ]:
endpoint_name = "<Endpoint Name>"
endpoints = tc.endpoint_search(endpoint_name, fields="id,display_name")
endpoint_id = None

for ep in endpoints:
    if ep['display_name'].upper() == endpoint_name.upper():
        endpoint_id = ep['id']
        break

if endpoint_id is None:
    print(f"Error: endpoint with name '{endpoint_name}' not found")
else:
    print(f"ID of endpoint with name'{endpoint_name}': {endpoint_id}")

### Step 4: Transfer a file

In [ ]:
source_endpoint = "<Source Endpoint UUID>"  #Your local endpoint
dest_endpoint = "<Destination Endpoint UUID>" #Endpoint shared with you

tc.endpoint_autoactivate(source_endpoint)
tc.endpoint_autoactivate(dest_endpoint)

source_path = "<Source Path>" #Copy this from Globus File Manager
dest_path = "<Destination Path>"

filename = "Test.txt"

tdata = globus_sdk.TransferData(tc, source_endpoint, dest_endpoint)

tdata.add_item(source_path + filename, dest_path + filename)

submit_result = tc.submit_transfer(tdata)
print(f"Task ID: {submit_result['task_id']}")

### Step 5: Get the status of the task

In [ ]:
status = tc.get_task(submit_result['task_id'], fields="status")['status']
print(f"Task completed with status {status}")